In [1]:
import warnings
import pandas as pd
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.ml.feature import Bucketizer

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

/home/kartik/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
spark = SparkSession.builder.master("local[*]").appName("PySpark-Churn").getOrCreate()

24/02/25 16:05:51 WARN Utils: Your hostname, kartik-ASUS-TUF-Gaming-A15-FA506IC-FA506IC resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
24/02/25 16:05:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 16:05:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/25 16:05:52 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: a9:7e:f9:40:c1:91:89:d4


In [3]:
spark

In [4]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=PySpark-Churn>

In [6]:
spark_df =spark.read.format("csv").option("header", "true").option("inferschema","true").load("/home/kartik/Downloads/prsnl_project/Churn_Predictions.csv")
spark_df.dtypes

[('RowNumber', 'int'),
 ('CustomerId', 'int'),
 ('Surname', 'string'),
 ('CreditScore', 'int'),
 ('Geography', 'string'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Tenure', 'int'),
 ('Balance', 'double'),
 ('NumOfProducts', 'int'),
 ('HasCrCard', 'int'),
 ('IsActiveMember', 'int'),
 ('EstimatedSalary', 'double'),
 ('Exited', 'int')]

In [7]:
spark_df.head()

Row(RowNumber=1, CustomerId=15634602, Surname='Hargrave', CreditScore=619, Geography='France', Gender='Female', Age=42, Tenure=2, Balance=0.0, NumOfProducts=1, HasCrCard=1, IsActiveMember=1, EstimatedSalary=101348.88, Exited=1)

In [8]:
# Count for the columns
print("Shape: ", (spark_df.count(), len(spark_df.columns)))

# Variable types
spark_df.printSchema()
spark_df.dtypes

Shape:  (10000, 14)
root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



[('RowNumber', 'int'),
 ('CustomerId', 'int'),
 ('Surname', 'string'),
 ('CreditScore', 'int'),
 ('Geography', 'string'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Tenure', 'int'),
 ('Balance', 'double'),
 ('NumOfProducts', 'int'),
 ('HasCrCard', 'int'),
 ('IsActiveMember', 'int'),
 ('EstimatedSalary', 'double'),
 ('Exited', 'int')]

In [9]:
# Show the Age column
spark_df.select(spark_df.Age).show()

+---+
|Age|
+---+
| 42|
| 41|
| 42|
| 39|
| 43|
| 44|
| 50|
| 29|
| 44|
| 27|
| 31|
| 24|
| 34|
| 25|
| 35|
| 45|
| 58|
| 24|
| 45|
| 24|
+---+
only showing top 20 rows



In [10]:
# Lowercase columns
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])
spark_df.describe(["age", "exited"]).show()

+-------+------------------+-------------------+
|summary|               age|             exited|
+-------+------------------+-------------------+
|  count|             10000|              10000|
|   mean|           38.9218|             0.2037|
| stddev|10.487806451704587|0.40276858399486065|
|    min|                18|                  0|
|    max|                92|                  1|
+-------+------------------+-------------------+



In [11]:
# Group by for the churn column
spark_df.groupby("exited").count().show()

+------+-----+
|exited|count|
+------+-----+
|     1| 2037|
|     0| 7963|
+------+-----+



In [16]:
spark_df.groupby("exited").agg({"age": "mean"}).show()

+------+-----------------+
|exited|         avg(age)|
+------+-----------------+
|     1| 44.8379970544919|
|     0|37.40838879819164|
+------+-----------------+



In [17]:
# Select numerical columns check the statistics
num_cols = [col[0] for col in spark_df.dtypes if col[1] != 'string']
spark_df.select(num_cols).describe().show()
spark_df.select(num_cols).describe().toPandas().transpose()

24/02/25 16:14:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-----------------+-------------------+
|summary|         rownumber|       customerid|      creditscore|               age|            tenure|          balance|     numofproducts|          hascrcard|     isactivemember|  estimatedsalary|             exited|
+-------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-----------------+-------------------+
|  count|             10000|            10000|            10000|             10000|             10000|            10000|             10000|              10000|              10000|            10000|              10000|
|   mean|            5000.5|  1.56909405694E7|         650.5288|           38.9218|            5.0128|76485.88928799961|        

,0,1,2,3,4
summary,count,mean,stddev,min,max
rownumber,10000,5000.5,2886.8956799071675,1,10000
customerid,10000,1.56909405694E7,71936.18612274907,15565701,15815690
creditscore,10000,650.5288,96.65329873613035,350,850
age,10000,38.9218,10.487806451704587,18,92
tenure,10000,5.0128,2.8921743770496837,0,10
balance,10000,76485.88928799961,62397.40520238599,0.0,250898.09
numofproducts,10000,1.5302,0.5816543579989917,1,4
hascrcard,10000,0.7055,0.45584046447513327,0,1
isactivemember,10000,0.5151,0.49979692845891815,0,1


In [18]:
# Select categorical columns and describe the statistics
cat_cols = [col[0] for col in spark_df.dtypes if col[1] == 'string']

for col in cat_cols:
    spark_df.select(col).distinct().show()

+------------+
|     surname|
+------------+
|       Tyler|
|     Palermo|
|      Piccio|
|    Lazareva|
|  Kambinachi|
|       Virgo|
| Baryshnikov|
|     Wofford|
|      Lavrov|
|   Bezrukova|
|      Avdeev|
|      Clunie|
|      Duigan|
|    Sokolova|
|      Azarov|
|    Rawlings|
|         Zox|
|       Rubeo|
|      Arbour|
|Rapuluchukwu|
+------------+
only showing top 20 rows

+---------+
|geography|
+---------+
|  Germany|
|   France|
|    Spain|
+---------+

+------+
|gender|
+------+
|Female|
|  Male|
+------+



In [19]:
# Numerical columns statistics based on churn data
for col in num_cols:
    spark_df.groupby("exited").agg({col: "mean"}).show()

+------+-----------------+
|exited|   avg(rownumber)|
+------+-----------------+
|     1|4905.917525773196|
|     0|5024.694964209469|
+------+-----------------+

+------+--------------------+
|exited|     avg(customerid)|
+------+--------------------+
|     1|1.5690051964653904E7|
|     0|1.5691167881702876E7|
+------+--------------------+

+------+-----------------+
|exited| avg(creditscore)|
+------+-----------------+
|     1|645.3514972999509|
|     0|651.8531960316463|
+------+-----------------+

+------+-----------------+
|exited|         avg(age)|
+------+-----------------+
|     1| 44.8379970544919|
|     0|37.40838879819164|
+------+-----------------+

+------+-----------------+
|exited|      avg(tenure)|
+------+-----------------+
|     1|4.932744231713304|
|     0|5.033278914981791|
+------+-----------------+

+------+-----------------+
|exited|     avg(balance)|
+------+-----------------+
|     1|91108.53933726063|
|     0|72745.29677885193|
+------+-----------------+

+---

In [20]:
spark_df.createOrReplaceTempView("tbl_df")

spark.sql("show databases").show()

spark.sql("show tables").show()

spark.sql("select age from tbl_df limit 5").show()

spark.sql("select exited, avg(age) from tbl_df group by exited").show()


+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tbl_df|       true|
+---------+---------+-----------+

+---+
|age|
+---+
| 42|
| 41|
| 42|
| 39|
| 43|
+---+

+------+-----------------+
|exited|         avg(age)|
+------+-----------------+
|     1| 44.8379970544919|
|     0|37.40838879819164|
+------+-----------------+



In [21]:
############################
# Feature Interaction
############################
from pyspark.sql.functions import when, count, col

# Filling missing values with 50
spark_df = spark_df.fillna(50)

# segment a and segment b for using the product higher than 5 years
spark_df = spark_df.withColumn('segment', 
                               when(spark_df['tenure'] < 5, "segment_b").otherwise("segment_a"))
# credit score range
spark_df = spark_df.withColumn('credit_range',
                    when(spark_df['creditscore'] < 300, 1).
                    when((spark_df['creditscore'] >= 300) & (spark_df['creditscore'] < 501), 2).
                    when((spark_df['creditscore'] >= 501) & (spark_df['creditscore'] < 601), 3).
                    when((spark_df['creditscore'] >= 601) & (spark_df['creditscore'] < 701), 4).
                    when((spark_df['creditscore'] >= 701) & (spark_df['creditscore'] < 801), 5).
                    otherwise(6))
# Estimated retirement for diffrent countries
spark_df = spark_df.withColumn('est_retirement',
                    when((spark_df['age'] >= 65) & (spark_df['geography'] == "Germany"), 1).
                    when((spark_df['age'] < 65) & (spark_df['geography'] == "Germany"), 0).
                    when((spark_df['age'] >= 66) & (spark_df['geography'] == "Spain"), 1).
                    when((spark_df['age'] < 66) & (spark_df['geography'] == "Spain"), 0).
                    when((spark_df['age'] >= 62) & (spark_df['geography'] == "France"), 1).
                    when((spark_df['age'] < 62) & (spark_df['geography'] == "France"), 0))

# Tenure / Number of Product
spark_df = spark_df.withColumn('tenure_over_nop', spark_df.tenure / spark_df.numofproducts)
# Credit Score / Estimated Salary
spark_df = spark_df.withColumn('cs_over_salary', spark_df.creditscore / spark_df.estimatedsalary)
# Estimated Salary / Tenure
spark_df = spark_df.withColumn('estsalary_over_tenure', spark_df.estimatedsalary / spark_df.tenure)
# Balance is equal to 0 or Not
spark_df = spark_df.withColumn('balance0',
                    when(spark_df['balance'] > 0.0, 0).otherwise(1))

############################
# Bucketization / Bining / Num to Cat
############################

bucketizer = Bucketizer(splits=[18, 30, 40, 50, 60, 92], inputCol="age", outputCol="age_cat")

spark_df = bucketizer.setHandleInvalid("keep").transform(spark_df)

spark_df = spark_df.withColumn('age_cat', spark_df.age_cat + 1)

spark_df = spark_df.drop("balance", "creditscore", "rownumber", "customerid", "surname", "estimatedsalary")

spark_df.show(20)

+---------+------+---+------+-------------+---------+--------------+------+---------+------------+--------------+------------------+--------------------+---------------------+--------+-------+
|geography|gender|age|tenure|numofproducts|hascrcard|isactivemember|exited|  segment|credit_range|est_retirement|   tenure_over_nop|      cs_over_salary|estsalary_over_tenure|balance0|age_cat|
+---------+------+---+------+-------------+---------+--------------+------+---------+------------+--------------+------------------+--------------------+---------------------+--------+-------+
|   France|Female| 42|     2|            1|        1|             1|     1|segment_b|           4|             0|               2.0|0.006107615594765329|             50674.44|       1|    3.0|
|    Spain|Female| 41|     1|            1|        0|             1|     0|segment_b|           4|             0|               1.0|0.005402399696186101|            112542.58|       0|    3.0|
|   France|Female| 42|     8|      

In [22]:
############################
# Label Encoding
############################

indexer = StringIndexer(inputCol="segment", outputCol="segment_label")

indexer.fit(spark_df).transform(spark_df)

temp_sdf = indexer.fit(spark_df).transform(spark_df)

spark_df = temp_sdf.withColumn("segment_label", temp_sdf["segment_label"].cast("integer"))

spark_df = spark_df.drop('segment')

# Geography column label encoding

indexer = StringIndexer(inputCol="geography", outputCol="country_label")

indexer.fit(spark_df).transform(spark_df)

temp_sdf = indexer.fit(spark_df).transform(spark_df)

spark_df = temp_sdf.withColumn("country_label", temp_sdf["country_label"].cast("integer"))

spark_df = spark_df.drop('geography')

In [23]:
############################
# One Hot Encoding
############################
encoder = OneHotEncoder(inputCols=["age_cat"], outputCols=["age_cat_ohe"])
spark_df = encoder.fit(spark_df).transform(spark_df)

spark_df.show(5)


+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+
|gender|age|tenure|numofproducts|hascrcard|isactivemember|exited|credit_range|est_retirement|   tenure_over_nop|      cs_over_salary|estsalary_over_tenure|balance0|age_cat|segment_label|country_label|  age_cat_ohe|
+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+
|Female| 42|     2|            1|        1|             1|     1|           4|             0|               2.0|0.006107615594765329|             50674.44|       1|    3.0|            1|            0|(5,[3],[1.0])|
|Female| 41|     1|            1|        0|             1|     0|           4|             0|               1.0|0.005402399696186101|       

In [24]:
############################
# Defining TARGET
############################

stringIndexer = StringIndexer(inputCol='exited', outputCol='label')
temp_sdf = stringIndexer.fit(spark_df).transform(spark_df)
spark_df = temp_sdf.withColumn("label", temp_sdf["label"].cast("integer"))
spark_df.show(5)

+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+-----+
|gender|age|tenure|numofproducts|hascrcard|isactivemember|exited|credit_range|est_retirement|   tenure_over_nop|      cs_over_salary|estsalary_over_tenure|balance0|age_cat|segment_label|country_label|  age_cat_ohe|label|
+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+-----+
|Female| 42|     2|            1|        1|             1|     1|           4|             0|               2.0|0.006107615594765329|             50674.44|       1|    3.0|            1|            0|(5,[3],[1.0])|    1|
|Female| 41|     1|            1|        0|             1|     0|           4|             0|               1.0|0.00

In [29]:
############################
# Defining Features
############################

cols = ['age', 'tenure', 'numofproducts', 'hascrcard',
        'isactivemember', 'segment_label', 'age_cat_ohe', 'credit_range', 'est_retirement', 'tenure_over_nop', 'cs_over_salary',
       'estsalary_over_tenure', 'balance0', 'country_label']
va = VectorAssembler(inputCols=cols, outputCol="features",handleInvalid="skip")
va_df = va.transform(spark_df)
va_df.show()


+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+-----+--------------------+
|gender|age|tenure|numofproducts|hascrcard|isactivemember|exited|credit_range|est_retirement|   tenure_over_nop|      cs_over_salary|estsalary_over_tenure|balance0|age_cat|segment_label|country_label|  age_cat_ohe|label|            features|
+------+---+------+-------------+---------+--------------+------+------------+--------------+------------------+--------------------+---------------------+--------+-------+-------------+-------------+-------------+-----+--------------------+
|Female| 42|     2|            1|        1|             1|     1|           4|             0|               2.0|0.006107615594765329|             50674.44|       1|    3.0|            1|            0|(5,[3],[1.0])|    1|[42.0,2.0,1.0,1.0...|
|Female| 41|     1|            1

In [30]:
final_df = va_df.select("features", "label")
final_df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[42.0,2.0,1.0,1.0...|    1|
|[41.0,1.0,1.0,0.0...|    0|
|(18,[0,1,2,3,9,11...|    1|
|(18,[0,1,2,5,8,11...|    0|
|[43.0,2.0,1.0,1.0...|    0|
+--------------------+-----+
only showing top 5 rows



In [31]:
# Split the dataset into test and train sets.
train_df, test_df = final_df.randomSplit([0.7, 0.3], seed=17)
train_df.show(10)
test_df.show(10)

print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(18,[0,1,2,3,4,5,...|    0|
|(18,[0,1,2,3,4,5,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
+--------------------+-----+
only showing top 10 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(18,[0,1,2,3,4,5,...|    1|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
|(18,[0,1,2,3,4,7,...|    0|
+--------------------+-----+
only showing top 10 rows

Training Dataset Count: 6673
Test Dataset Count: 2914


In [32]:
############################
# Gradient Boosted Tree Classifier
############################
gbm = GBTClassifier(maxIter=100, featuresCol="features", labelCol="label")
gbm_model = gbm.fit(train_df)
y_pred = gbm_model.transform(test_df)
y_pred.show(5)

y_pred.filter(y_pred.label == y_pred.prediction).count() / y_pred.count()



+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(18,[0,1,2,3,4,5,...|    1|[-1.9824549809097...|[0.01861659253233...|       1.0|
|(18,[0,1,2,3,4,7,...|    0|[1.54441313220974...|[0.95642947917751...|       0.0|
|(18,[0,1,2,3,4,7,...|    0|[1.46061456492074...|[0.94888594651194...|       0.0|
|(18,[0,1,2,3,4,7,...|    0|[1.03003880703091...|[0.88696195178223...|       0.0|
|(18,[0,1,2,3,4,7,...|    0|[1.19971130062871...|[0.91678326336734...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



0.8665065202470831

In [33]:
############################
# Model Tuning
############################

evaluator = BinaryClassificationEvaluator()


gbm_params = (ParamGridBuilder()
              .addGrid(gbm.maxDepth, [2, 4, 6])
              .addGrid(gbm.maxBins, [20, 30])
              .addGrid(gbm.maxIter, [10, 20])
              .build())



cv = CrossValidator(estimator=gbm,
                    estimatorParamMaps=gbm_params,
                    evaluator=evaluator,
                    numFolds=5)

cv_model = cv.fit(train_df)


y_pred = cv_model.transform(test_df)
ac = y_pred.select("label", "prediction")
ac.filter(ac.label == ac.prediction).count() / ac.count()

0.8634179821551132